In [4]:
# Set up access credentials
consumer_info = [line.strip() for line in open("df_twitter_consumer_auth.txt")]
access_info = [line.strip() for line in open("df_twitter_access_auth.txt")]


In [90]:
#authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
new_tweets = api.user_timeline(screen_name = "CremeDeLaCrypto",count=10)

In [85]:
search_terms = ["bitcoin","blockchain","crypto","cryptocurrency","btc","ethereum"]

for item in t:
    if any(term in item.text.lower() for term in search_terms):
        print("yes")

yes
yes
yes
yes
yes


In [82]:
t[9].text.lower()

'excited about crypto?!\n\ninvesting *financial capital* is just one way to go long the industry. \n\nanother is to inve… https://t.co/ksrnysdt5j'

In [92]:
a = []

#save most recent tweets
b = [item for item in new_tweets if any(term in item.text.lower() for term in search_terms)]


In [101]:
len(b)

5

In [102]:
# Original source: https://gist.github.com/yanofsky/5436496

import tweepy #https://github.com/tweepy/tweepy
import csv
import datetime

#Twitter API credentials
consumer_key = consumer_info[0]
consumer_secret = consumer_info[1]
access_key = access_info[0] 
access_secret = access_info[1]


def get_all_tweets(screen_name,datetime_limit, search_terms):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []    
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #Filter and save most recent tweets
    filtered_tweets = [item for item in new_tweets if any(term in item.text.lower() for term in search_terms)]
    alltweets.extend(filtered_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while (len(new_tweets) > 0):
            
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print (screen_name + " " +str(alltweets[-1].created_at))

        if (alltweets[-1].created_at <= datetime_limit):
            print ("Reached the limit " + str(datetime_limit))
            print (screen_name + " : %s tweets downloaded" % (len(alltweets)))
            break
            
          
    #transform the tweepy tweets into a 2D array that will populate the csv    
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
    
    return outtweets



In [ ]:
# tweets = get_all_tweets("Cointelegraph")

In [103]:
# Source: https://www.reddit.com/r/Bitcoin/comments/6m6g0c/who_are_some_good_people_to_follow_on_twitter_for/

twitter_accounts = ["FINTECHcircle","cburniske","niccary","CremeDeLaCrypto","Cointelegraph","aantonop"]
search_terms = ["bitcoin","blockchain","crypto","cryptocurrency","btc","ethereum","litecoin","bch","eth"]
datetime_limit = datetime.datetime(2017, 1, 1, 0, 0, 0)

tweets = []
for account in twitter_accounts:
    tmp = get_all_tweets(account,datetime_limit,search_terms)
    tweets.extend(tmp)


FINTECHcircle 2017-11-06 16:48:10
FINTECHcircle 2017-10-25 03:45:01
FINTECHcircle 2017-10-04 07:02:31
FINTECHcircle 2017-09-15 11:37:04
FINTECHcircle 2017-08-17 09:41:43
FINTECHcircle 2017-07-18 11:45:10
FINTECHcircle 2017-06-28 12:15:59
FINTECHcircle 2017-06-24 16:07:21
FINTECHcircle 2017-06-06 08:40:01
FINTECHcircle 2017-05-15 15:55:47
FINTECHcircle 2017-04-25 16:25:21
FINTECHcircle 2017-04-04 12:19:04
FINTECHcircle 2017-03-13 12:43:14
FINTECHcircle 2017-02-23 11:39:23
FINTECHcircle 2017-02-01 07:04:05
FINTECHcircle 2017-01-31 08:11:55
FINTECHcircle 2017-01-31 08:11:55
cburniske 2017-11-19 19:00:00
cburniske 2017-11-06 14:17:44
cburniske 2017-10-21 16:22:19
cburniske 2017-10-05 20:15:57
cburniske 2017-09-21 11:09:05
cburniske 2017-09-05 21:22:39
cburniske 2017-08-25 22:23:54
cburniske 2017-08-14 02:47:47
cburniske 2017-08-02 20:05:44
cburniske 2017-07-20 17:52:22
cburniske 2017-07-10 11:49:08
cburniske 2017-07-02 17:43:23
cburniske 2017-06-21 19:16:07
cburniske 2017-06-14 12:46:24
cb

In [105]:
len(tweets)

13345

In [129]:
import pandas as pd

tw_export = pd.DataFrame({
    "tweet":[item[2].decode("utf-8") for item in tweets],
    "timestamp":[item[1].strftime("%Y-%m-%d %H:%M:%S") for item in tweets]
})
tw_export.head()

tw_export[["tweet","timestamp"]].to_csv("data/tweepy_tweets.csv",index=False)